In [17]:
import re
import requests
from bs4 import BeautifulSoup
from collections import defaultdict

In [11]:
RACE_URL = "https://ballotpedia.org/North_Carolina_gubernatorial_election,_2020"
race_page = requests.get(RACE_URL)

In [16]:
race_soup = BeautifulSoup(race_page.content, "html.parser")
voteboxes = race_soup.find_all("div", {"class": "votebox"})

In [22]:
cands = defaultdict(lambda: {"races": []})
for elem in voteboxes:
    race = elem.find("h5", {"class": "votebox-header-election-type"}).get_text().strip()
    for td in elem.find_all("td", {"class": "votebox-results-cell--text"}):
        a = td.find("a")
        name = a.get_text().strip()
        url = a["href"]  # URL as unique key
        cands[url]["name"] = name
        cands[url]["races"].append(race)

print(cands)

defaultdict(<function <lambda> at 0x7fdf572bdaf0>, {'https://ballotpedia.org/Roy_Cooper': {'races': ['General election for Governor of North Carolina', 'Democratic primary for Governor of North Carolina'], 'name': 'Roy Cooper'}, 'https://ballotpedia.org/Dan_Forest': {'races': ['General election for Governor of North Carolina', 'Republican primary for Governor of North Carolina'], 'name': 'Dan Forest'}, 'https://ballotpedia.org/Steven_DiFiore_II': {'races': ['General election for Governor of North Carolina'], 'name': 'Steven DiFiore  II'}, 'https://ballotpedia.org/Al_Pisano': {'races': ['General election for Governor of North Carolina'], 'name': 'Al Pisano'}, 'https://ballotpedia.org/Ernest_Reeves': {'races': ['Democratic primary for Governor of North Carolina'], 'name': 'Ernest Reeves'}, 'https://ballotpedia.org/Holly_Grange': {'races': ['Republican primary for Governor of North Carolina'], 'name': 'Holly Grange'}})


In [5]:
CAND_URL = "https://ballotpedia.org/Roy_Cooper"
cand_page = requests.get(CAND_URL)

In [6]:
cand_soup = BeautifulSoup(cand_page.content, "html.parser")
widgets = cand_soup.find_all("div", {"class": "widget-row"})

In [10]:
edu = {}
edu_found = False
for elem in widgets:
    if edu_found:
        if "value-only" in elem["class"]:
            break
        else:
            degree = elem.find("div", {"class": "widget-key"}).get_text().strip()
            school = elem.find("div", {"class": "widget-value"}).get_text().strip()
            edu[degree] = school
    elif elem.find("p", text=re.compile("\s*Education\s*")):
        edu_found = True

print(edu)

{"Bachelor's": 'University of North Carolina, Chapel Hill, 1979', 'Law': 'University of North Carolina School of Law, 1982'}
